In [226]:
import pandas as pd
import numpy as np

In [227]:
# 파일 불러오기
fifaDF = pd.read_csv('C:/Users/ss/Desktop/11주_4조/data/FIFA_train.csv', encoding='utf-8')
fifaDF_test = pd.read_csv('C:/Users/ss/Desktop/11주_4조/data/FIFA_test.csv', encoding='utf-8')

In [228]:
# 필요없는 데이터 삭제
fifaDF = fifaDF.iloc[:,2:]
fifaDF_test = fifaDF_test.iloc[:,2:]
fifaDF.head()

,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0
2,31,south america,2021,ST,right,5.0,91,91,3.0,80000000.0
3,32,europe,2020,DF,right,4.0,91,91,3.0,51000000.0
4,25,europe,2021,GK,right,3.0,90,93,1.0,68000000.0


In [229]:
# 계약기간 값 정렬
fifaDF.contract_until.value_counts()

2019            2366
2021            2308
2020            2041
2022             761
2023             506
Jun 30, 2019     501
2018             327
Dec 31, 2018      64
May 31, 2019      19
2024              12
Jan 31, 2019      10
Jun 30, 2020       9
2025               3
Jan 1, 2019        2
2026               1
May 31, 2020       1
Jan 12, 2019       1
Name: contract_until, dtype: int64

In [230]:
# 계약 연도만 추출
fifaDF['contract_until'] = fifaDF['contract_until'].map(lambda x : x[-4:])
fifaDF_test['contract_until'] = fifaDF_test['contract_until'].map(lambda x : x[-4:])

In [231]:
fifaDF['contract_until'].unique()

array(['2021', '2020', '2019', '2023', '2022', '2024', '2026', '2018',
       '2025'], dtype=object)

In [232]:
fifaDF_importance = fifaDF.iloc[:,[0,3,6,7,8,9]]

In [233]:
fifaDF_test_importance = fifaDF_test.iloc[:,[0,3,6,7,8]]

In [234]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [235]:
le = LabelEncoder()

for column_name in fifaDF_importance.columns:
    if fifaDF_importance[column_name].dtype == object:
        fifaDF_importance[column_name] = le.fit_transform(fifaDF_importance[column_name])
    else:
        pass
    

C:\Users\ss\AppData\Local\Temp\ipykernel_13884\2419833201.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifaDF_importance[column_name] = le.fit_transform(fifaDF_importance[column_name])


In [236]:
for column_name in fifaDF_test_importance.columns:
    if fifaDF_test_importance[column_name].dtype == object:
        fifaDF_test_importance[column_name] = le.fit_transform(fifaDF_test_importance[column_name])
    else:
        pass

C:\Users\ss\AppData\Local\Temp\ipykernel_13884\3739740088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifaDF_test_importance[column_name] = le.fit_transform(fifaDF_test_importance[column_name])


In [237]:
# data, target 분리
data = fifaDF_importance.iloc[:,:-1]
target = fifaDF_importance.iloc[:,-1]

In [238]:
data

,age,position,stat_overall,stat_potential,stat_skill_moves
0,31,3,94,94,4.0
1,27,1,91,93,1.0
2,31,3,91,91,3.0
3,32,0,91,91,3.0
4,25,1,90,93,1.0
...,...,...,...,...,...
8927,18,2,48,63,3.0
8928,19,0,47,59,2.0
8929,18,0,47,64,2.0
8930,18,1,47,65,1.0


In [239]:
from sklearn.model_selection import train_test_split

In [240]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [241]:
train_input = train_input.to_numpy()
test_input = test_input.to_numpy()

In [242]:
ss = StandardScaler()
ss.fit(train_input)
train_input_scaled = ss.fit_transform(train_input)
test_input_scaled = ss.transform(test_input)

In [245]:
from sklearn.utils import all_estimators

In [246]:
models = all_estimators(type_filter='regressor')

In [247]:
# 경고무시
import warnings
warnings.filterwarnings('ignore')

RMSE_list = []
for name, model in models:
    try:
        # 모델 객체 생성
        md = model()
        # 학습
        md.fit(train_input_scaled, train_target)
        # 평가
        RMSE = np.sqrt(mean_squared_error(test_target, md.predict(test_input_scaled)))
        RMSE_list.append((name, RMSE))        
    except :
        pass

In [253]:
RMSE_list

[('GradientBoostingRegressor', 1017344.9257581142),
 ('ExtraTreeRegressor', 1101386.4994494624),
 ('ExtraTreesRegressor', 1155696.3595360797),
 ('DecisionTreeRegressor', 1239067.3888481515),
 ('RandomForestRegressor', 1263299.5276848727),
 ('BaggingRegressor', 1302420.6071230692),
 ('HistGradientBoostingRegressor', 1553875.2593544975),
 ('KNeighborsRegressor', 1862582.0178889248),
 ('PoissonRegressor', 2280151.100057382),
 ('AdaBoostRegressor', 2325295.545243294),
 ('GammaRegressor', 4357129.535112962),
 ('OrthogonalMatchingPursuitCV', 4913252.0237943735),
 ('Lars', 4913252.023794374),
 ('LarsCV', 4913252.023794374),
 ('LassoLarsCV', 4913252.023794374),
 ('LassoLarsIC', 4913252.023794374),
 ('LinearRegression', 4913252.023794374),
 ('TransformedTargetRegressor', 4913252.023794374),
 ('Lasso', 4913252.099671987),
 ('LassoLars', 4913258.214942823),
 ('Ridge', 4913307.31550499),
 ('RidgeCV', 4913307.315506349),
 ('BayesianRidge', 4913504.342411249),
 ('LassoCV', 4913554.503063267),
 ('SGD

In [254]:
from sklearn.preprocessing import PolynomialFeatures

In [255]:
pmf = PolynomialFeatures(degree=2, include_bias=False)

In [256]:
poly_train_input = pmf.fit_transform(train_input_scaled)
poly_test_input = pmf.transform(test_input_scaled)

In [263]:
# 경고무시
import warnings
warnings.filterwarnings('ignore')

RMSE_list = []
for name, model in models:
    if name == 'GradientBoostingRegressor' :
        try:
        # 모델 객체 생성
            md = model()
        # 학습
            md.fit(train_input_scaled, train_target)
        # 평가
            RMSE = np.sqrt(mean_squared_error(test_target, md.predict(test_input_scaled)))
            RMSE_list.append((name, RMSE))        
        except :
            pass
    else:
        pass

In [264]:
RMSE_list

[('GradientBoostingRegressor', 1036771.031334635)]

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [46]:
gbr = GradientBoostingRegressor()
gbr2 = GradientBoostingRegressor()

In [42]:
gbr.fit(poly_train_input,train_target)

GradientBoostingRegressor()

In [48]:
gbr2.fit(train_input_scaled,train_target)

GradientBoostingRegressor()

In [44]:
print(gbr.score(poly_train_input, train_target))
print(gbr.score(poly_test_input, test_target))

0.9963646180324469
0.9741725322887738


In [ ]:
param_grid = {'learning_rate' : [0.01, 0.1, 1, 10],
              'n_estimators' : [100, 500, 1000],
              'max_depth' : [3, 5, 7, 10]}

In [171]:
grid_search = GridSearchCV(gbr, param_grid, cv=5, return_train_score=True)

In [172]:
grid_search.fit(poly_train_input, train_target)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: Ru

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py:231: RuntimeWarning: overflow encountered in square
  * np.sum(sample_weight * ((y - raw_predictions.ravel()) ** 2))
C:\ProgramData\Anaconda3\lib\site-packages\sk

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 1, 10],
                         'n_estimators': [100, 500, 1000]},
             return_train_score=True)

In [174]:
# 최적 test 값
print(f'best score :', grid_search.best_score_)
print(f'best parameter :', grid_search.best_params_)

best score : 0.9869753578179787
best parameter : {'learning_rate': 0.1, 'n_estimators': 1000}


In [176]:
gbrmodel = grid_search.best_estimator_
print(f'train score :', gbrmodel.score(poly_train_input, train_target))
print(f'test score :', gbrmodel.score(poly_test_input, test_target))

train score : 0.9991016691070905
test score : 0.9773589339787097


In [177]:
test = ss.transform(fifaDF_test_importance)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [178]:
test.shape

(3828, 5)

In [180]:
# test data predict
result = grid_search.best_estimator_
result_gbr = result.predict(pmf.transform(test))

In [181]:
sub = pd.read_csv('C:/Users/ss/Desktop/11주_4조/data/submission.csv', encoding='utf-8')

In [182]:
result_gbr.shape

(3828,)

In [183]:
sub['value'] = result_gbr

In [184]:
sub.to_csv('gbrpoly_imp모델.csv',index = False)

In [72]:
from sklearn.linear_model import LinearRegression

In [73]:
lr = LinearRegression()

In [76]:
lr.fit(poly_train_input, train_target)

LinearRegression()

In [79]:
print(lr.score(poly_train_input, train_target))
print(lr.score(poly_test_input, test_target))

0.8759005279340228
0.8449105572672264


In [80]:
from sklearn.linear_model import Ridge

In [152]:
ridge = Ridge(alpha=0.1, tol= 1)

In [153]:
ridge.fit(poly_train_input, train_target)

Ridge(alpha=0.1, tol=1)

In [154]:
print(ridge.score(poly_train_input, train_target))
print(ridge.score(poly_test_input, test_target))

0.8759005258040623
0.8449092274260845


In [155]:
param_grid2 = {'alpha' : [0.01, 0.1, 1, 10],
              'tol' : [0.001, 0.01, 0.01, 0.1, 1]}

In [156]:
grid_search = GridSearchCV(ridge, param_grid2, cv=5, return_train_score=True)

In [157]:
grid_search.fit(poly_train_input, train_target)

GridSearchCV(cv=5, estimator=Ridge(alpha=0.1, tol=1),
             param_grid={'alpha': [0.01, 0.1, 1, 10],
                         'tol': [0.001, 0.01, 0.01, 0.1, 1]},
             return_train_score=True)

In [158]:
ridgemodel = grid_search.best_estimator_
print(f'train score :', ridgemodel.score(poly_train_input, train_target))
print(f'test score :', ridgemodel.score(poly_test_input, test_target))

train score : 0.8759003205605833
test score : 0.8448970073085806


In [159]:
grid_search.best_score_

0.8739242036254868

In [166]:
result_ridge = ridgemodel.predict(pmf.transform(test))

In [167]:
sub2 = pd.read_csv('C:/Users/ss/Desktop/11주_4조/data/submission.csv', encoding='utf-8')

In [168]:
sub2['value'] = result_ridge

In [169]:
sub2.to_csv('ridge_imp모델.csv',index = False)

In [170]:
sub2

,id,value
0,1,5.195503e+07
1,2,5.336181e+07
2,4,4.728726e+07
3,5,4.831302e+07
4,6,4.136055e+07
...,...,...
3823,16924,7.755333e+06
3824,16929,6.876018e+06
3825,16932,6.109172e+06
3826,16937,7.456652e+06


In [188]:
targetdf = pd.read_csv('C:/Users/ss/Desktop/11주_4조/data/FIFA_train.csv', encoding='utf-8')

In [189]:
targetdf = targetdf.iloc[:,2:]

In [192]:
targetdf = fifaDF.iloc[:,[0,3,6,7,8]]

In [200]:
targetdf

array([[31.,  3., 94., 94.,  4.],
       [27.,  1., 91., 93.,  1.],
       [31.,  3., 91., 91.,  3.],
       ...,
       [18.,  0., 47., 64.,  2.],
       [18.,  1., 47., 65.,  1.],
       [19.,  3., 47., 63.,  2.]])

In [194]:
for column_name in targetdf.columns:
    if targetdf[column_name].dtype == object:
        targetdf[column_name] = le.fit_transform(targetdf[column_name])
    else:
        pass

C:\Users\ss\AppData\Local\Temp\ipykernel_13884\1468082797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targetdf[column_name] = le.fit_transform(targetdf[column_name])


In [195]:
targetdf = targetdf.to_numpy()

In [203]:
targetdf = ss.transform(targetdf)

In [186]:
from sklearn.metrics import *

In [198]:
ffdf = pd.read_csv('C:/Users/ss/Desktop/11주_4조/data/FIFA_train(test용).csv', encoding='utf-8')

In [199]:
for column_name in ffdf.columns:
    if ffdf[column_name].dtype == object:
        ffdf[column_name] = le.fit_transform(ffdf[column_name])
    else:
        pass